In [ ]:
pip install azure-storage-blob

In [1]:
#De nodige imports om het script te laten werken worden geïmporteerd.
import os
from azure.storage.blob import BlobServiceClient, __version__
from multiprocessing.pool import ThreadPool
import pathlib

#De connectiestring die werd opgeslagen als systeem variable, wordt opgevraagd.
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

#We declareren de verschillende weken waarvan de foto's moeten worden opgehaald. En de paden waar de foto's moeten worden opgeslagen worden gedeclareerd.
weken = ["modelbotanic"]
local_path = "C:/Users/yorbe/Documents/School/3ITF/AI/Lessons/Project4.0-Algoritme/Data/"
rootfolder = "C:/Users/yorbe/Documents/School/3ITF/AI/Lessons/Project4.0-Algoritme/Data"



#We maken een klasse aan waarmee we de foto's uit de database kunnen halen.
class AzureBlobFileDownloader():
  #We verbinden het script met de databank en gaan in een container. (Bv mais-week-1)
  def __init__(self, week):
    self.blob_service_client =  BlobServiceClient.from_connection_string(connect_str)
    self.my_container = self.blob_service_client.get_container_client(week)
 
  #We lijsten alle foto's binnen een container op.
  def download_all_blobs_in_container(self):
    my_blobs = self.my_container.list_blobs()
    result = self.run(my_blobs)
 
  #We zorgen ervoor dat er 10 foto's per seconde gedownload worden.
  def run(self,blobs):
    with ThreadPool(processes=int(10)) as pool:
     return pool.map(self.save_blob_locally, blobs)
 
  #We declareren het pad waar de foto's gedownload moeten worden.
  def download_path(self, folder):
    return str(folder)

  #We slagen de foto's effectief op.
  def save_blob_locally(self, blob):
    file_name = blob.name
    bytes = self.my_container.get_blob_client(blob).download_blob().readall()
 
    download_file_path = os.path.join(self.download_path(folder), file_name)

    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
 
    with open(download_file_path, "wb") as file:
      file.write(bytes)
    return file_name
 

#We maken een while loop om alle weken te kunnen downloaden
y=0
while y <= len(weken)-1:
  #We zetten de paden goed, en maken mappen aan per week.
  os.chdir(rootfolder)

  if not os.path.exists(weken[y]):
    os.makedirs(weken[y])

  folder = pathlib.Path().resolve() / weken[y]

  print("Afbeeldingen van week", y+1, "worden gedownload.")

  #De klasse die we net hebben aangemaakt wordt aangesproken en de download paden worden gedeclareerd.
  azure_blob_file_downloader = AzureBlobFileDownloader(weken[y])
  azure_blob_file_downloader.download_all_blobs_in_container()
  azure_blob_file_downloader.download_path(folder)
  y +=1


print("Alle afbeeldingen zijn gedownload.")


Afbeeldingen van week 1 worden gedownload.
Alle afbeeldingen zijn gedownload.


In [ ]:
!pip install -Uqq fastbook


In [ ]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
path = "C:/Users/yorbe/Documents/School/3ITF/AI/Lessons/Project4.0-Algoritme/Data/"

In [ ]:
from fastai.vision.all import *
filenames = get_image_files(path)
print(filenames)

In [ ]:
failed = verify_images(filenames)
print(failed)

In [ ]:
groeistadias = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
dls = groeistadias.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=4, nrows=1)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
groeistadias = groeistadias.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = groeistadias.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1)

In [ ]:
groeistadias= cnn_learner(dls, resnet101, metrics=error_rate)
groeistadias.fit_one_cycle(2, 3e-3)
groeistadias.unfreeze()
groeistadias.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))
groeistadias.fine_tune(2, freeze_epochs=1)


In [ ]:
interp = ClassificationInterpretation.from_learner(groeistadias)
interp.plot_confusion_matrix()

In [ ]:
groeistadias.save('groeistadias')
groeistadias.export()